In [18]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True) 

GITHUB_TOKENS = os.getenv("TOKENS", "").split(",")

In [30]:
GITHUB_API_URL = 'https://api.github.com'

GITHUB_LIMIT_REQUESTS_BY_TOKEN_PER_HOUR = 5000
GITHUB_LIMIT_REQUESTS_BY_IP_PER_HOUR = 60

In [24]:
import requests

def request (path):
    response = requests.get(f'https://api.github.com/{path}')
    return response

In [ ]:
response = request('/users')

if response.status_code == 200:
    print("Success!")
    print(response.json())
else:
    print(f"Request failed with status code {response.status_code}")

Success!
[{'login': 'mojombo', 'id': 1, 'node_id': 'MDQ6VXNlcjE=', 'avatar_url': 'https://avatars.githubusercontent.com/u/1?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/mojombo', 'html_url': 'https://github.com/mojombo', 'followers_url': 'https://api.github.com/users/mojombo/followers', 'following_url': 'https://api.github.com/users/mojombo/following{/other_user}', 'gists_url': 'https://api.github.com/users/mojombo/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/mojombo/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/mojombo/subscriptions', 'organizations_url': 'https://api.github.com/users/mojombo/orgs', 'repos_url': 'https://api.github.com/users/mojombo/repos', 'events_url': 'https://api.github.com/users/mojombo/events{/privacy}', 'received_events_url': 'https://api.github.com/users/mojombo/received_events', 'type': 'User', 'user_view_type': 'public', 'site_admin': False}, {'login': 'defunkt', 'id': 2, 'node_id': 'MDQ6VXNlcj

In [32]:
for i in range(1, GITHUB_LIMIT_REQUESTS_BY_IP_PER_HOUR):
    response = request('/users')
    if response.status_code == 200:
        print(f"Request {i} succeeded!")
    else:
        print(f"Request {i} failed with status code {response.status_code}")
        break

Request 1 failed with status code 403
